In [ ]:
import torchgeo
import terratorch
import yaml
import pdb

In [ ]:
import torch
# from torchgeo.trainers import ObjectDetectionTask
# from terratorch.tasks import ObjectDetectionTask
from torchgeo.datasets import VHR10
from torch.utils.data import DataLoader
import lightning.pytorch as pl
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
import torch
import albumentations as A
from collections.abc import Callable
from functools import partial
from albumentations.pytorch import transforms as T
import numpy as np

class Normalize(Callable):
    def __init__(self, means, stds, max_pixel_value=None):
        super().__init__()
        self.means = means
        self.stds = stds

    def __call__(self,  **kwargs):
        # min_value = self.means - 2 * self.stds
        # max_value = self.means + 2 * self.stds
        # img = (batch["image"] - min_value) / (max_value - min_value)
        # img = torch.clip(img, 0, 1)
        # batch["image"] = img
        # return batch
        pdb.set_trace()
        batch = kwargs
        batch['image']=torch.stack(tuple(batch["image"]))
        image = batch["image"]/max_pixel_value if max_pixel_value is not None else batch["image"]
        if len(image.shape) == 5:
            means = torch.tensor(self.means, device=image.device).view(1, -1, 1, 1, 1)
            stds = torch.tensor(self.stds, device=image.device).view(1, -1, 1, 1, 1)
        elif len(image.shape) == 4:
            means = torch.tensor(self.means, device=image.device).view(1, -1, 1, 1)
            stds = torch.tensor(self.stds, device=image.device).view(1, -1, 1, 1)
        else:
            msg = f"Expected batch to have 5 or 4 dimensions, but got {len(image.shape)}"
            raise Exception(msg)
        batch["image"] = (image - means) / stds
        # pdb.set_trace()
        return batch


def get_transform(train, image_size=1472):
    transforms = []
    transforms.append(A.PadIfNeeded(min_height=image_size, min_width=image_size, value=0, border_mode=0))
    if train:
        transforms.append(A.CenterCrop(width=image_size, height=image_size))
        transforms.append(A.HorizontalFlip(p=0.5))
    else:
        transforms.append(A.CenterCrop(width=image_size, height=image_size))
    transforms.append(A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255))
    transforms.append(A.ToFloat())
    transforms.append(T.ToTensorV2())
    # return A.Compose(transforms, additional_targets={'boxes': 'bboxes', 'masks': 'mask'})
    return A.Compose(transforms, bbox_params=A.BboxParams(format="pascal_voc", label_fields=['labels']), is_check_shapes=False)

def apply_transforms(sample, transforms):


    # pdb.set_trace()
    sample['image']=torch.stack(tuple(sample["image"]))
    sample['image'] = sample['image'].permute(1, 2, 0) if len(sample['image'].shape) == 3 else sample['image'].permute(0, 2, 3, 1)
    sample['image'] = np.array(sample['image'].cpu())
    sample["masks"] = [np.array(torch.stack(tuple(x)).cpu()) for x in sample["masks"]]
    # sample["masks"] = np.array(sample["masks"].cpu())
    sample["boxes"] = np.array(sample["boxes"].cpu())
    sample["labels"] = np.array(sample["labels"].cpu())
    # sample["boxes"] = [torch.stack(tuple(x)) for x in sample["masks"]]
    # sample["labels"] =  
    transformed = transforms(image=sample['image'],
                             masks=sample["masks"], 
                             # bboxes=np.array(torch.stack(tuple(sample["boxes"]), dim=0).cpu()), 
                             bboxes=sample["boxes"],
                             labels=sample["labels"])
    
    transformed['boxes'] = torch.tensor(transformed['bboxes'])
    transformed['labels'] = torch.tensor(transformed['labels'], dtype=torch.int64)
    masks_before = len(transformed['masks'])
    transformed['masks'] = [x for x in transformed['masks'] if x.any()]
    masks_after = len(transformed['masks'])
    if masks_before != masks_after:
        print("Dropped masks:", str(masks_before - masks_after))
    del transformed['bboxes']
    # print("Done transform")
    return transformed

transforms = get_transform(True, image_size=896)

In [ ]:
from terratorch.datasets import mVHR10 as mVHR10Dataset
from terratorch.datamodules import mVHR10DataModule

In [ ]:
def preprocess(sample):
    sample["image"] = sample["image"].float() / 255.0
    return sample
# ds = VHR10(root='data/VHR10/', split='positive', transforms=preprocess, download=True)
ds_old = VHR10(root='data/VHR10/', split='positive', transforms=partial(apply_transforms, transforms=transforms), download=True)
ds = mVHR10Dataset(second_level_split = 'test', root = 'data/VHR10/', transforms=partial(apply_transforms, transforms=transforms))

In [ ]:
ds = mVHR10Dataset(second_level_split = 'train', root = 'data/VHR10/', transforms=partial(apply_transforms, transforms=transforms), second_level_split_proportions=(1, 0, 0))

In [ ]:
ds_old = VHR10(root='data/VHR10/', split='positive', download=True)


In [ ]:
import pandas as pd
df_faulty = pd.DataFrame()

# ds_old = VHR10(root='data/VHR10/', split='positive', download=True)

# for i in range(len(ds)):
    
#     masks_n = len(ds[i]['masks'])
#     boxes_n = len(ds[i]['boxes'])
#     labels_n = len(ds[i]['labels'])
    
#     if ((masks_n != boxes_n) | (labels_n != boxes_n) | (labels_n != masks_n)):
#         df_tmp = pd.DataFrame({'id': [i], 'masks_n': [masks_n], 'boxes_n': [boxes_n], 'labels_n': [labels_n]})
#         df_faulty = pd.concat([df_faulty, df_tmp], axis=0)
        
for i in range(len(ds)):
    
    masks_n = len(ds[i]['masks'])
    boxes_n = len(ds[i]['boxes'])
    labels_n = len(ds[i]['labels'])
    print(i)
    if ((masks_n != boxes_n) | (labels_n != boxes_n) | (labels_n != masks_n)):
        print(i)
        df_tmp = pd.DataFrame({'id': [i], 'masks_n': [masks_n], 'boxes_n': [boxes_n], 'labels_n': [labels_n]})
        df_faulty = pd.concat([df_faulty, df_tmp], axis=0)
    print("#############")
        

In [ ]:
df_faulty

In [ ]:
ds.plot(ds[85])
plt.show()

In [ ]:
for i in range(20):
    ds.plot(ds_old[i])
    plt.show()

In [ ]:
def collate_fn(batch):
    new_batch = {
        "image": [item["image"] for item in batch],
        "boxes": [item["boxes"] for item in batch],
        "labels": [item["labels"] for item in batch],
        "masks": [item["masks"] for item in batch],
    }
    return new_batch

# dl = DataLoader(ds_old, batch_size=16, num_workers=16, shuffle=True, collate_fn=collate_fn)

In [ ]:
# The current (torchgeo 0.5.0) ObjectDetectionTask does not seem to support variable
# size inputs. We can quickly fix this by subclassing it and overriding the
# training_step method.
# from torchgeo.trainers import ObjectDetectionTask
from terratorch.tasks import ObjectDetectionTask

class VariableSizeInputObjectDetectionTask(ObjectDetectionTask):
    def training_step(self, batch, batch_idx, dataloader_idx=0):

        x = batch["image"]
        batch_size = len(x)  # we change this line to support variable size inputs
        y = [
            {"boxes": batch["boxes"][i], "labels": batch["labels"][i]}
            for i in range(batch_size)
        ]
        loss_dict = self(x, y)
        # pdb.set_trace()
        if isinstance(loss_dict, dict) == False:
            loss_dict = loss_dict.output
        train_loss: Tensor = sum(loss_dict.values())
        self.log("train_loss", train_loss)
        if torch.isnan(train_loss):
            pdb.set_trace()
        #self.log_dict(loss_dict)
        return train_loss



In [ ]:
# task = VariableSizeInputObjectDetectionTask(
#     model="fcos",
#     backbone="resnet18",
#     weights=True,
#     in_channels=3,
#     num_classes=11,
#     trainable_layers=3,
#     lr=1e-3,
#     patience=10,
#     freeze_backbone=False,
# )

# tb_logger = TensorBoardLogger(save_dir="../../../logs/", name="torchgeo")

In [ ]:
# with open('../confs/object_detection_vhr10_prithvi.yaml', 'r') as file:
with open('../confs/object_detection_vhr10_resnet50.yaml', 'r') as file:
# with open('../confs/object_detection_vhr10_vit.yaml', 'r') as file:    
#with open('../confs/object_detection_vhr10_Clay.yaml', 'r') as file:    
    config = yaml.safe_load(file)

# framework = 'faster-rcnn' 
framework = 'mask-rcnn'
# framework = 'fcos'
# framework = 'retinanet'

config['model']['init_args']['model_args']['framework'] = framework

In [ ]:
config['model']['init_args']['model_args']

In [ ]:
config['model']

In [ ]:
task = ObjectDetectionTask(
    optimizer=config['optimizer']['class_path'].split('.')[-1],
    optimizer_hparams=config['optimizer']['init_args'],
    scheduler=config['lr_scheduler']['class_path'].split('.')[-1],
    scheduler_hparams=config['lr_scheduler']['init_args'],
    # patience=10,
    **config['model']['init_args']
)

# tb_logger = TensorBoardLogger(save_dir="../../../logs/", name=config['trainer']['logger']['init_args']['name'])
tb_logger = TensorBoardLogger(save_dir="../../../logs/", name=framework + '_test')

In [ ]:
# %debug

In [ ]:
task.monitor


In [ ]:
# task.monitor = "loss_classifier"
# task.monitor = "train_loss"

In [ ]:
trainer = pl.Trainer(
    default_root_dir="logs/",
    accelerator="gpu",
    devices=[0],
    # accelerator="cpu",
    min_epochs=0,
    max_epochs=2,
    log_every_n_steps=20,
    logger=tb_logger
)

In [ ]:
use_cuda = torch.cuda.is_available()
print(f"GPU available: {use_cuda}")

In [ ]:
# datamodule = mVHR10DataModule(root='data/VHR10/', batch_size=8, second_level_split_proportions=(0.9, 0.05, 0.05))
datamodule = mVHR10DataModule(root='data/VHR10/', batch_size=8)

In [ ]:
trainer.fit(task, datamodule=datamodule)

In [ ]:
%debug

## Inference example

In [ ]:
# path = "/opt/app-root/src/fm-geospatial/pf/logs/terratorch_datamodule/version_7/checkpoints/epoch=28-step=2146.ckpt"
path = "/opt/app-root/src/fm-geospatial/pf/logs/terratorch/version_9/checkpoints/epoch=99-step=4100.ckpt"

In [ ]:
state_dict = torch.load(path)

In [ ]:
state_dict = state_dict['state_dict']

In [ ]:
new_state_dict = {}
for key in state_dict.keys():
    new_key = key.replace('model.torchvision', 'torchvision')  # Replace with your specific changes
    new_state_dict[new_key] = state_dict[key]

In [ ]:
state_dict

In [ ]:
new_state_dict

In [ ]:
model = task.model

In [ ]:
model.state_dict()

In [ ]:
model.load_state_dict(new_state_dict)

In [ ]:
model

In [ ]:
from torchvision.ops import nms

def apply_nms(boxes, scores, labels, iou_threshold=0.1, score_threshold=0.1):
    # Filter based on score threshold
    keep_score = scores > score_threshold
    boxes, scores, labels = boxes[keep_score], scores[keep_score], labels[keep_score]
    
    # Apply NMS
    keep_nms = nms(boxes, scores, iou_threshold)
    
    return boxes[keep_nms], scores[keep_nms], labels[keep_nms]

In [ ]:
datamodule.setup('test')
datamodule.setup('fit')

In [ ]:
%debug

In [ ]:
test_dl = datamodule._dataloader_factory('test')
# train_dl = datamodule._dataloader_factory('train')

In [ ]:
batch = next(iter(test_dl))
# batch = next(iter(train_dl))
# batch = next(iter(dl))
#batch["image"] = [image.to("cuda:0") for image in batch["image"]]

In [ ]:
batch = datamodule.aug(batch)

In [ ]:
model.eval()

with torch.no_grad():
    out = model(batch["image"])

In [ ]:
try:
    output_list = out.output
except:
    print("No wrapper")

In [ ]:
for i, out in enumerate(output_list):
    print(str(i), '   ###################################')
    print(out)
 

In [ ]:
for batch_idx in range(len(batch["image"])):

    sample = {
        "image": batch["image"][batch_idx],
        "boxes": batch["boxes"][batch_idx],
        "labels": batch["labels"][batch_idx],
        "masks": batch["masks"][batch_idx],
        "prediction_labels":  output_list[batch_idx]["labels"],
        "prediction_boxes":  output_list[batch_idx]["boxes"],
        "prediction_scores": output_list[batch_idx]["scores"],
        "prediction_masks": output_list[batch_idx]["masks"],
    }
    ds.plot(sample)
    plt.show()
    plt.close()

# sample["prediction_boxes"], sample["prediction_scores"],sample["prediction_labels"]  = apply_nms(sample["prediction_boxes"], sample["prediction_scores"],sample["prediction_labels"])

In [ ]:
ds.plot(sample)
plt.show()
plt.close()